In [1]:
import tensorflow as tf

In [2]:
x=tf.Variable(3,name="x")
y=tf.Variable(4,name="y")
f=x*x*y+y+2


In [4]:
with tf.Session() as sess:
    x.initializer.run()
    y.initializer.run()
    result=f.eval()
result

42

In [6]:
# the shortcut
init=tf.global_variables_initializer()
with tf.Session() as sess:
    init.run()
    result=f.eval()
result

42

In [8]:
# linear regression
import numpy as np
from sklearn.datasets import fetch_california_housing

In [12]:
housing=fetch_california_housing()
m,n=housing.data.shape
housing_data_plus_bias=np.c_[np.ones((m,1)),housing.data]
(m,n)

(20640, 8)

In [10]:
X=tf.constant(housing_data_plus_bias,dtype=tf.float32,name="X")
y=tf.constant(housing.target.reshape(-1,1),dtype=tf.float32,name="y")
XT=tf.transpose(X)
theta=tf.matmul(tf.matmul(tf.matrix_inverse(tf.matmul(XT,X)),XT),y)

In [11]:
with tf.Session() as sess:
    theta_value=theta.eval()
theta_value

array([[-3.7185181e+01],
       [ 4.3633747e-01],
       [ 9.3952334e-03],
       [-1.0711310e-01],
       [ 6.4479220e-01],
       [-4.0338000e-06],
       [-3.7813708e-03],
       [-4.2348403e-01],
       [-4.3721911e-01]], dtype=float32)